In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from _My_Functions import null_cols, rmse

importing Jupyter notebook from _My_Functions.ipynb


In [2]:
acc = pd.read_csv('acc_1_train_basicimputation_predum.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,0,13.99,123.03,3,14,10.0,MORTGAGE,55000.0,0,Dec-2015,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,148.0,677.0,-1.0,-1.0
1,20000.0,20000.0,20000.0,0,9.17,637.58,2,7,10.0,MORTGAGE,180000.0,0,Dec-2015,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,306.0,682.0,-1.0,-1.0
2,20000.0,20000.0,20000.0,0,8.49,631.26,2,6,10.0,MORTGAGE,85000.0,0,Dec-2015,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,202.0,707.0,-1.0,-1.0
3,10000.0,10000.0,10000.0,0,6.49,306.45,1,2,6.0,RENT,85000.0,0,Dec-2015,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,164.0,687.0,-1.0,-1.0
4,8000.0,8000.0,8000.0,0,11.48,263.74,2,10,10.0,MORTGAGE,42000.0,0,Dec-2015,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,253.0,702.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Data Cleaning and Imputation

## Add ROI

In [3]:
acc['roi'] = (acc['total_pymnt'] - acc['funded_amnt']) / acc['funded_amnt']

acc['roi']

0         0.228257
1         0.069690
2         0.076925
3         0.099897
4         0.117448
            ...   
674045   -0.540367
674046   -0.744641
674047   -0.737395
674048   -0.930236
674049   -0.883189
Name: roi, Length: 674050, dtype: float64

In [4]:
# copying dataset just in case
acc_clean = copy.deepcopy(acc)

# Home Ownership

In [5]:
# Assigning values to home_ownership status

acc_clean.loc[acc_clean['home_ownership']=='OWN', 'home_ownership'] = 3

acc_clean.loc[acc_clean['home_ownership']=='MORTGAGE', 'home_ownership'] = 2

acc_clean.loc[acc_clean['home_ownership']=='ANY', 'home_ownership'] = 2

acc_clean.loc[acc_clean['home_ownership']=='OTHER', 'home_ownership'] = 1

acc_clean.loc[acc_clean['home_ownership']=='RENT', 'home_ownership'] = 0

acc_clean.loc[acc_clean['home_ownership']=='NONE', 'home_ownership'] = 0

acc_clean['home_ownership'] = acc_clean['home_ownership'].astype(int)

# Grade

In [6]:
sorted(acc_clean['grade'].unique())

[1, 2, 3, 4, 5, 6, 7]

In [7]:
# Grouping E, F, and G into F

acc_clean['grouped_grade_NO_EG'] = acc_clean['grade']

acc_clean.loc[acc_clean['grouped_grade_NO_EG'] == 6, 'grouped_grade_NO_EG'] = 5
acc_clean.loc[acc_clean['grouped_grade_NO_EG'] == 7, 'grouped_grade_NO_EG'] = 5

In [8]:
# replacing letter grades with numbers for grouped grades

# get sorted list of grades
grade_list = sorted(list(acc_clean['grouped_grade_NO_EG'].unique()))

i = 1

for grades in grade_list:
    acc_clean.loc[acc_clean['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
    i+=1

acc_clean['grouped_grade_NO_EG'] = acc_clean['grouped_grade_NO_EG'].astype(int)
acc_clean['grouped_grade_NO_EG'].unique()

array([3, 2, 1, 5, 4])

## Subgrade

In [9]:
sorted(acc_clean['sub_grade'].unique())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35]

In [10]:
# converting back to alphanumreric for ease of feature engineering below
dic_x = {1:'A', 2:'B', 3:'C', 4:'D', 5:'E', 6:'F', 7:'G'}
for x in sorted(acc_clean['sub_grade'].unique()):
    if x % 5 == 0:
        letr = (x // 5)
        numr = 5
    else:
        letr = (x // 5) + 1
        numr = x % 5
    acc_clean.loc[acc_clean['sub_grade']==x, 'sub_grade'] = dic_x[letr]+str(numr)

In [11]:
acc_clean['sub_grade']

0         C4
1         B2
2         B1
3         A2
4         B5
          ..
674045    C1
674046    B4
674047    C3
674048    C4
674049    C4
Name: sub_grade, Length: 674050, dtype: object

In [12]:
# grouping subgrades 1, 2 vs 3, 4, 5

acc_clean['grouped_subgrade_12_345'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('2',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('3',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('4',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('5',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'].value_counts()

B2    124582
C2    108651
C1     82310
A2     77262
B1     72570
D2     52458
D1     48213
A1     40268
E2     24347
E1     22538
F1      8641
F2      7577
G1      2594
G2      2039
Name: grouped_subgrade_12_345, dtype: int64

In [13]:
# grouping subgrades 1, 2, 3 vs 4, 5

acc_clean['grouped_subgrade_123_45'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('2',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('3',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('4',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('5',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_123_45'].value_counts()

C1    119841
B1    113642
B2     83510
C2     71120
D1     67803
A1     59295
A2     58235
D2     32868
E1     31770
E2     15115
F1     11737
F2      4481
G1      3417
G2      1216
Name: grouped_subgrade_123_45, dtype: int64

In [14]:
# grouping Subgrades

acc_clean['grouped_subgrade'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].str.replace('E', 'F', regex=False)

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].str.replace('G', 'F', regex=False)

In [15]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade_12_345'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
    i+=1

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].astype(int)
acc_clean['grouped_subgrade_12_345'].unique()

array([ 6,  3,  1,  4,  5, 10,  2,  7,  8, 11,  9, 12, 14, 13])

In [16]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade_123_45'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
    i+=1

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].astype(int)
acc_clean['grouped_subgrade_123_45'].unique()

array([ 6,  3,  1,  4,  5,  9,  2,  7, 11, 10,  8, 12, 14, 13])

In [17]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
    i+=1

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].astype(int)
acc_clean['grouped_subgrade'].unique()

array([14,  7,  6,  2, 10, 11, 23, 13,  4,  1,  9, 15, 16,  8, 18, 12, 22,
       17,  5, 24, 19, 20,  3, 21, 25])

## Reducing Income if not Verified

In [18]:
acc_clean['annual_inc']

0          55000.0
1         180000.0
2          85000.0
3          85000.0
4          42000.0
            ...   
674045     62000.0
674046     49400.0
674047    125000.0
674048     95000.0
674049    157000.0
Name: annual_inc, Length: 674050, dtype: float64

In [19]:
acc_clean['annual_inc_reduced'] = acc_clean['annual_inc']

acc_clean.loc[acc_clean['verification_status']==0,\
              'annual_inc_reduced'] = acc_clean['annual_inc'] * 0.7

In [20]:
acc_clean['annual_inc_joint_reduced'] = acc_clean['annual_inc_joint']

acc_clean.loc[acc_clean['verification_status_joint']==0,
                  'annual_inc_joint_reduced'] = acc_clean['annual_inc_joint'] * 0.7

In [21]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

""
0
1
2
3
4
...
674045
674046
674047
674048


In [22]:
cols_w_null_list = list(acc_clean[cols_w_null].columns)
cols_w_null_list

[]

In [23]:
# getting the columns where there are 0s

for col in cols_w_null_list:
    acc_clean.loc[acc_clean[col].isnull(), col] = -1
    
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,roi,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced
0,3600.0,3600.0,3600.0,0,13.99,123.03,3,C4,10.0,2,55000.0,0,Dec-2015,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,148.0,677.0,-1.0,-1.0,0.228257,3,6,6,14,38500.0,-1.0
1,20000.0,20000.0,20000.0,0,9.17,637.58,2,B2,10.0,2,180000.0,0,Dec-2015,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,306.0,682.0,-1.0,-1.0,0.069690,2,3,3,7,126000.0,-1.0
2,20000.0,20000.0,20000.0,0,8.49,631.26,2,B1,10.0,2,85000.0,0,Dec-2015,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,202.0,707.0,-1.0,-1.0,0.076925,2,3,3,6,59500.0,-1.0
3,10000.0,10000.0,10000.0,0,6.49,306.45,1,A2,6.0,0,85000.0,0,Dec-2015,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,164.0,687.0,-1.0,-1.0,0.099897,1,1,1,2,59500.0,-1.0
4,8000.0,8000.0,8000.0,0,11.48,263.74,2,B5,10.0,2,42000.0,0,Dec-2015,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,253.0,702.0,-

In [24]:
acc_clean[acc_clean['pct_tl_nvr_dlq'].isnull()]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,roi,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced


In [25]:
num_cols = [col for col in acc_clean.columns if '2' in col]

acc_clean[num_cols]

,delinq_2yrs,collections_12_mths_ex_med,acc_open_past_24mths,chargeoff_within_12_mths,grouped_subgrade_12_345,grouped_subgrade_123_45
0,0.0,0.0,4.0,0.0,6,6
1,0.0,0.0,6.0,0.0,3,3
2,1.0,0.0,4.0,0.0,3,3
3,0.0,0.0,7.0,0.0,1,1
4,0.0,0.0,5.0,0.0,4,4
...,...,...,...,...,...,...
674045,1.0,0.0,3.0,0.0,5,5
674046,0.0,0.0,12.0,0.0,4,4
674047,1.0,0.0,6.0,0.0,6,5
674048,0.0,0.0,3.0,0.0,6,6


In [26]:
num_cols.remove('grouped_subgrade_12_345')
num_cols.remove('grouped_subgrade_123_45')

num_cols

['delinq_2yrs',
 'collections_12_mths_ex_med',
 'acc_open_past_24mths',
 'chargeoff_within_12_mths']

In [27]:
acc_clean = acc_clean.drop(num_cols, axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,roi,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced
0,3600.0,3600.0,3600.0,0,13.99,123.03,3,C4,10.0,2,55000.0,0,Dec-2015,debt_consolidation,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,148.0,677.0,-1.0,-1.0,0.228257,3,6,6,14,38500.0,-1.0
1,20000.0,20000.0,20000.0,0,9.17,637.58,2,B2,10.0,2,180000.0,0,Dec-2015,debt_consolidation,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,306.0,682.0,-1.0,-1.0,0.069690,2,3,3,7,126000.0,-1.0
2,20000.0,20000.0,20000.0,0,8.49,631.26,2,B1,10.0,2,85000.0,0,Dec-2015,major_purchase,17.61,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,17700.0,13674.0,5.7,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,202.0,707.0,-1.0,-1.0,0.076925,2,3,3,6,59500.0,-1.0
3,10000.0,10000.0,10000.0,0,6.49,306.45,1,A2,6.0,0,85000.0,0,Dec-2015,credit_card,13.07,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,1997.0,8182.0,50.1,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,164.0,687.0,-1.0,-1.0,0.099897,1,1,1,2,59500.0,-1.0
4,8000.0,8000.0,8000.0,0,11.48,263.74,2,B5,10.0,2,42000.0,0,Dec-2015,credit_card,34.80,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,28528.0,9966.0,41.4,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,253.0,702.0,-1.0,-1.0,0.117448,2,4,4,10,29400.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Income : * Ratios

In [28]:
acc_clean[(acc_clean['application_type']==1) &\
              (acc_clean['annual_inc_joint']==acc_clean['annual_inc'])]\
[['annual_inc_joint', 'annual_inc']]

,annual_inc_joint,annual_inc
384338,-1.0,-1.0
384339,-1.0,-1.0
384366,-1.0,-1.0


In [29]:
acc_clean[acc_clean['application_type']==2]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,roi,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced
157,15000.0,15000.0,15000.0,0,11.99,498.15,3,C1,1.0,2,65000.0,1,Dec-2015,debt_consolidation,17.56,54.0,10000.0,9.0,0.0,13028.0,88.3,16.0,w,15183.460000,10000.0,1.0,2,77000.0,15.40,0,0.0,0.0,231860.0,0.0,30.0,22525.0,74.0,2839.0,78.0,14800.0,3.0,1.0,25762.0,295.0,94.2,0.0,99.0,136.0,13.0,13.0,1.0,28.0,10000.0,11.0,54.0,2.0,7.0,2.0,3.0,4.0,7.0,11.0,7.0,9.0,92.3,100.0,0.0,0.0,251645.0,35553.0,5100.0,30649.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,136.0,667.0,-1.0,-1.0,0.012231,3,5,5,11,65000.0,53900.0
186,24000.0,24000.0,24000.0,1,23.13,678.37,6,F2,6.0,2,43435.0,1,Dec-2015,debt_consolidation,35.70,10000.0,84.0,13.0,1.0,12508.0,73.1,22.0,w,27488.251245,10000.0,1.0,2,113314.0,25.44,0,0.0,156.0,182215.0,6.0,10.0,48582.0,75.0,5690.0,74.0,17100.0,1.0,2.0,14017.0,357.0,96.8,0.0,52.0,209.0,13.0,10.0,1.0,26.0,10000.0,12.0,10000.0,2.0,5.0,2.0,4.0,9.0,6.0,12.0,5.0,13.0,100.0,100.0,1.0,0.0,208064.0,61090.0,11000.0,61739.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,209.0,682.0,-1.0,-1.0,0.145344,5,11,11,22,43435.0,79319.8
193,24000.0,24000.0,24000.0,1,13.44,551.50,3,C3,1.0,2,96000.0,1,Dec-2015,credit_card,22.70,55.0,63.0,13.0,1.0,19403.0,53.3,24.0,w,30785.080592,55.0,1.0,2,117000.0,22.76,0,0.0,0.0,220879.0,4.0,10.0,43145.0,70.0,6210.0,64.0,36400.0,2.0,0.0,16991.0,6328.0,60.9,0.0,158.0,71.0,11.0,10.0,2.0,19.0,10000.0,6.0,55.0,2.0,5.0,3.0,4.0,8.0,8.0,13.0,5.0,13.0,95.7,66.7,0.0,0.0,258301.0,62548.0,16200.0,61251.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,174.0,677.0,-1.0,-1.0,0.282712,3,6,5,13,96000.0,81900.0
343,35000.0,35000.0,35000.0,1,17.97,888.20,4,D4,10.0,2,150000.0,0,Dec-2015,debt_consolidation,25.11,20.0,10000.0,27.0,0.0,39913.0,85.3,45.0,w,43193.305750,25.0,1.0,2,192000.0,25.28,0,0.0,433.0,603735.0,5.0,11.0,139765.0,74.0,13130.0,78.0,46800.0,3.0,8.0,22361.0,3761.0,89.4,0.0,165.0,404.0,0.0,0.0,7.0,29.0,20.0,0.0,20.0,9.0,15.0,10.0,15.0,9.0,18.0,27.0,16.0,27.0,84.4,80.0,0.0,0.0,733890.0,180363.0,35600.0,166148.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,404.0,672.0,-1.0,-1.0,0.234094,4,8,8,19,105000.0,134400.0
887,10500.0,10500.0,10500.0,0,9.80,337.83,2,B3,0.0,2,65000.0,1,Dec-2015,debt_consolidation,21.49,10.0,10000.0,29.0,0.0,11156.0,83.9,43.0,w,12150.121933,65.0,1.0,2,92000.0,15.85,0,0.0,0.0,350643.0,24.0,1.0,167875.0,96.0,4970.0,95.0,13300.0,0.0,1.0,12091.0,2144.0,83.9,0.0,127.0,287.0,17.0,1.0,1.0,17.0,10.0,2.0,10.0,4.0,4.0,4.0,6.0,36.0,4.0,6.0,4.0,29.0,93.0,50.0,0.0,0.0,373179.0,179031.0,13300.0,184122.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,287.0,667.0,-1.0,-1.0,0.157154,2,4,3,8,65000.0,64400.0
...,...,...,...,...,...,...,...,...,...,...,

In [30]:
acc_clean[acc_clean['annual_inc_joint']>0]['annual_inc_joint']

157        77000.0
186       113314.0
193       117000.0
343       192000.0
887        92000.0
            ...   
673887     80000.0
673905    132500.0
673976    143000.0
674009     50000.0
674016    108000.0
Name: annual_inc_joint, Length: 12796, dtype: float64

In [31]:
acc_clean['annual_inc_joint'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
674045    False
674046    False
674047    False
674048    False
674049    False
Name: annual_inc_joint, Length: 674050, dtype: bool

## Giving weight to loan amount

In [32]:
# creating a new column
acc_clean['weighted_loan_amnt'] = acc_clean['loan_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 1

acc_clean['weighted_loan_amnt']

0         10800.0
1         60000.0
2         40000.0
3         30000.0
4         16000.0
           ...   
674045    72000.0
674046    43200.0
674047    96000.0
674048    72800.0
674049    64000.0
Name: weighted_loan_amnt, Length: 674050, dtype: float64

## Giving weight to funded amount

In [33]:
# creating a new column
acc_clean['weighted_funded_amnt'] = acc_clean['funded_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 1

acc_clean['weighted_funded_amnt']

0         10800.0
1         60000.0
2         40000.0
3         30000.0
4         16000.0
           ...   
674045    72000.0
674046    43200.0
674047    96000.0
674048    72800.0
674049    64000.0
Name: weighted_funded_amnt, Length: 674050, dtype: float64

## Giving weight to installment

In [34]:
# creating a new column
acc_clean['weighted_installment'] = acc_clean['installment']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_installment'] = acc_clean['weighted_installment'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_installment'] = acc_clean['weighted_installment'] * 1

acc_clean['weighted_installment']

0          369.09
1         1912.74
2         1262.52
3          919.35
4          527.48
           ...   
674045    1630.50
674046     939.06
674047    2233.26
674048    1712.48
674049    1505.48
Name: weighted_installment, Length: 674050, dtype: float64

### 1st borrower income

In [35]:
# Income to loan amount ratio

acc_clean['ratio_inc_loan'] = acc_clean['annual_inc'] / acc_clean['loan_amnt']

acc_clean['ratio_inc_weightedloan'] = acc_clean['annual_inc'] / acc_clean['weighted_loan_amnt']


# income reduced to loan amount ratio

acc_clean['ratio_redinc_loan'] = acc_clean['annual_inc_reduced'] / acc_clean['loan_amnt']

acc_clean['ratio_redinc_weightedloan'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_loan_amnt']


# Income to funded amount ratio

acc_clean['ratio_inc_funded'] = acc_clean['annual_inc'] / acc_clean['funded_amnt']

acc_clean['ratio_inc_weightedfunded'] = acc_clean['annual_inc'] / acc_clean['weighted_funded_amnt']


# income reduced to funded amount ratio

acc_clean['ratio_redinc_funded'] = acc_clean['annual_inc_reduced'] /\
acc_clean['funded_amnt']

acc_clean['ratio_redinc_weightedfunded'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_funded_amnt']


# installment to income ratio

acc_clean['ratio_install_inc'] = acc_clean['annual_inc'] / acc_clean['installment']

acc_clean['ratio_weightedinstall_inc'] = acc_clean['annual_inc'] / acc_clean['weighted_installment']


# installment to reduced income ratio

acc_clean['ratio_install_redinc'] = acc_clean['annual_inc_reduced'] /\
acc_clean['installment']

acc_clean['ratio_weightedinstall_redinc'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_installment']

### joint borrower income

In [36]:
# Income to loan amount ratio

acc_clean['ratio_incjoint_loan'] = acc_clean['annual_inc_joint'] / acc_clean['loan_amnt']

acc_clean['ratio_incjoint_weightedloan'] = acc_clean['annual_inc_joint'] /\
acc_clean['weighted_loan_amnt']


# income reduced to loan amount ratio

acc_clean['ratio_redincjoint_loan'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['loan_amnt']

acc_clean['ratio_redincjoint_weightedloan'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['weighted_loan_amnt']


# Income to funded amount ratio

acc_clean['ratio_incjoint_funded'] = acc_clean['annual_inc_joint'] /\
acc_clean['funded_amnt']

acc_clean['ratio_incjoint_weightedfunded'] = acc_clean['annual_inc_joint'] /\
acc_clean['weighted_funded_amnt']


# income reduced to funded amount ratio

acc_clean['ratio_redincjoint_funded'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['funded_amnt']

acc_clean['ratio_redincjoint_weightedfunded'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['weighted_funded_amnt']


# Income to installment ratio

acc_clean['ratio_incjoint_installment'] = acc_clean['annual_inc_joint']/\
acc_clean['installment']

acc_clean['ratio_incjoint_weightedinstallment'] = acc_clean['annual_inc_joint']/\
acc_clean['weighted_installment']


# Reduced income to installment ratio

acc_clean['ratio_redincjoint_installment'] = acc_clean['annual_inc_joint_reduced']/\
acc_clean['installment']

acc_clean['ratio_redincjoint_weightedinstallment'] = acc_clean['annual_inc_joint_reduced']/\
acc_clean['weighted_installment']

# Loan Amount vs Funded Amount

In [37]:
acc_clean['loan_minus_funded'] = acc_clean['loan_amnt'] - acc_clean['funded_amnt']

acc_clean['ratio_loan_funded'] = acc_clean['loan_amnt'] / acc_clean['funded_amnt']

# Dummification

In [38]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_clean.select_dtypes(include=['object']).columns)

cols_w_obj

['sub_grade', 'issue_d', 'purpose', 'initial_list_status']

In [39]:
# dummification

acc_dum = pd.get_dummies(acc_clean, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,roi,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar

In [40]:
list(acc_dum.columns)

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'dti',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'total_pymnt',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_act_il',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',


# Scaling the dataset

In [41]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt', 'funded_amnt', 'roi']])

acc_dum = acc_dum.drop(['total_pymnt', 'funded_amnt', 'roi'], axis=1)

In [42]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [43]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue

In [44]:
col_names = list(acc_full.columns)
to_add = ['total_pymnt', 'funded_amnt', 'roi']

for col in to_add:
    col_names.append(col)

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue

## Downsampling the dataset

In [45]:
target_1b_train_downsample = pd.read_csv("target_1_train_basicimputation.csv.gz",
                                         compression='gzip',
                                         header=0,
                                         sep=',',
                                         quotechar='"')

In [46]:
target_1b_train_downsample[['loan_status_binary']]

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
674045,1
674046,1
674047,1
674048,1


In [47]:
from sklearn.utils import resample

label = target_1b_train_downsample['loan_status_binary']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False,
                         n_samples=int(len(target_1b_train_downsample.index)*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_full.iloc[combined]
target_subsample = target_1b_train_downsample[['loan_status_binary']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

loan_status_binary    0.5
dtype: float64

In [48]:
acc_subsample

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue

In [49]:
target_subsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
269355,1
269356,1
269357,1
269358,1


In [50]:
target_subsample.value_counts()

loan_status_binary
0                     134680
1                     134680
dtype: int64

In [51]:
target_subsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
269355,1
269356,1
269357,1
269358,1


In [52]:
acc_comparison = copy.deepcopy(acc_subsample[['funded_amnt', 'total_pymnt', 'roi']])

acc_comparison = pd.DataFrame(np.concatenate((acc_comparison, target_subsample), axis=1),
                              columns = ['funded_amnt', 'total_pymnt', 'roi', 'loan_status_binary'])

acc_comparison

,funded_amnt,total_pymnt,roi,loan_status_binary
0,28975.0,31330.651948,0.081299,0.0
1,22000.0,31555.128844,0.434324,0.0
2,19000.0,20286.408670,0.067706,0.0
3,20000.0,22862.968301,0.143148,0.0
4,1800.0,2035.138597,0.130633,0.0
...,...,...,...,...
269355,24000.0,11031.190000,-0.540367,1.0
269356,14400.0,3677.170000,-0.744641,1.0
269357,32000.0,8403.350000,-0.737395,1.0
269358,36400.0,2539.420000,-0.930236,1.0


In [53]:
acc_subsample_drop = acc_subsample.drop(['total_pymnt', 'roi'], axis=1)
acc_subsample_drop

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,sec_app_duration_of_credit_months,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,annual_inc_reduced,annual_inc_joint_reduced,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,issue_d_Apr-2009,issue_d_Apr-2010,issue_d_Apr-2011,issue_d_Apr-2012,issue_d_Apr-2013,issue_d_Apr-2014,issue_d_Apr-2015,issue_d_Apr-2016,issue_d_Apr-2017,issue_d_Apr-2018,issue_d_Aug-2007,issue_d_Aug-2008,issue_d_Aug-2009,issue_d_Aug-2010,issue_d_Aug-2011,issue_d_Aug-2012,issue_d_Aug-2013,issue_d_Aug-2014,issue_d_Aug-2015,issue_d_Aug-2016,issue_d_Aug-2017,issue_d_Aug-2018,issue_d_Dec-2007,issue_d_Dec-2008,issue_d_Dec-2009,issue_d_Dec-2010,issue_d_Dec-2011,issue_d_Dec-2012,issue_d_Dec-2013,issue_d_Dec-2014,issue_d_Dec-2015,issue_d_Dec-2016,issue_d_Dec-2017,issue_d_Dec-2018,issue_d_Feb-2008,issue_d_Feb-2009,issue_d_Feb-2010,issue_d_Feb-2011,issue_d_Feb-2012,issue_d_Feb-2013,issue_d_Feb-2014,issue_d_Feb-2015,issue_d_Feb-2016,issue_d_Feb-2017,issue_d_Feb-2018,issue_d_Jan-2008,issue_d_Jan-2009,issue_d_Jan-2010,issue_d_Jan-2011,issue_d_Jan-2012,issue_d_Jan-2013,issue_d_Jan-2014,issue_d_Jan-2015,issue_d_Jan-2016,issue_d_Jan-2017,issue_d_Jan-2018,issue_d_Jul-2007,issue_d_Jul-2008,issue_d_Jul-2009,issue_d_Jul-2010,issue_d_Jul-2011,issue_d_Jul-2012,issue_d_Jul-2013,issue_d_Jul-2014,issue_d_Jul-2015,issue_d_Jul-2016,issue_d_Jul-2017,issue_d_Jul-2018,issue_d_Jun-2007,issue_d_Jun-2008,issue_d_Jun-2009,issue_d_Jun-2010,issue_d_Jun-2011,issue_d_Jun-2012,issue_d_Jun-2013,issue_d_Jun-2014,issue_d_Jun-2015,issue_d_Jun-2016,issue_d_Jun-2017,issue_d_Jun-2018,issue_d_Mar-2008,issue_d_Mar-2009,issue_d_Mar-2010,issue_d_Mar-2011,issue_d_Mar-2012,issue_d_Mar-2013,issue_d_Mar-2014,issue_d_Mar-2015,issue

In [54]:
acc_subsample_drop.to_csv("acc_2_train_final.csv.gz",
                index=False,
                compression="gzip")

target_subsample.to_csv("target_2_train_final.csv.gz",
                        index=False,
                        compression="gzip")

In [55]:
acc_comparison.to_csv("acc_2_train_comparison.csv.gz",
                      index=False,
                      compression="gzip")

# Very Reduced Dataset

In [56]:
cols_w_purpose = [col for col in acc_subsample_drop.columns if 'purpose' in col]
acc_subsample[cols_w_purpose]

,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269355,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269357,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269358,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
cols_w_purpose = cols_w_purpose + ['annual_inc', 'fico_score', 'term', 'loan_amnt', 'home_ownership',
                                   'int_rate', 'all_util']

cols_w_purpose

['purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'annual_inc',
 'fico_score',
 'term',
 'loan_amnt',
 'home_ownership',
 'int_rate',
 'all_util']

In [58]:
acc_vred = copy.deepcopy(acc_subsample_drop[['annual_inc', 'fico_score', 'term', 'loan_amnt',
                                        'home_ownership', 'int_rate', 'all_util', 'dti']])


acc_vred.to_csv("acc_vred.csv.gz",
              index=False,
              compression="gzip")